# [Rinna-4Bのマルチモーダル会話モデルを試す](https://note.com/npaka/n/nf88f2523b4b1)

- 8/29現在でまだCPUで成功していません。GPU(RTX3070)はメモリオーバーです。
- 9/19現在で、A100環境でも実行していません。（他の作業がしたかったので後回しにしています。）

In [1]:
# パッケージのインストール
!git clone https://github.com/Vision-CAIR/MiniGPT-4.git
%cd ./MiniGPT-4
!git checkout 22d8888 # latest version as of July 31, 2023.
!wget https://huggingface.co/rinna/bilingual-gpt-neox-4b-minigpt4/resolve/main/customized_mini_gpt4.py
!wget https://huggingface.co/rinna/bilingual-gpt-neox-4b-minigpt4/resolve/main/checkpoint.pth


fatal: destination path 'MiniGPT-4' already exists and is not an empty directory.
/home/mhiavio/work/rinna/MiniGPT-4
HEAD is now at 22d8888 Update README.md
--2023-09-13 15:18:50--  https://huggingface.co/rinna/bilingual-gpt-neox-4b-minigpt4/resolve/main/customized_mini_gpt4.py
Resolving huggingface.co (huggingface.co)... 18.172.52.61, 18.172.52.38, 18.172.52.121, ...
Connecting to huggingface.co (huggingface.co)|18.172.52.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5964 (5.8K) [text/plain]
Saving to: ‘customized_mini_gpt4.py.1’

customized_mini_gpt 100%[===================>]   5.82K  --.-KB/s    in 0s      

2023-09-13 15:18:51 (879 MB/s) - ‘customized_mini_gpt4.py.1’ saved [5964/5964]

--2023-09-13 15:18:51--  https://huggingface.co/rinna/bilingual-gpt-neox-4b-minigpt4/resolve/main/checkpoint.pth
Resolving huggingface.co (huggingface.co)... 18.172.52.61, 18.172.52.38, 18.172.52.121, ...
Connecting to huggingface.co (huggingface.co)|18.172.52.61|:443..

In [2]:
!pip install accelerate==0.16.0
!pip install aiohttp==3.8.4
!pip install aiosignal==1.3.1
!pip install async-timeout==4.0.2
!pip install attrs==22.2.0
!pip install bitsandbytes==0.37.0
!pip install cchardet==2.1.7
!pip install chardet==5.1.0
!pip install contourpy==1.0.7
!pip install cycler==0.11.0
!pip install filelock==3.9.0
!pip install fonttools==4.38.0
!pip install frozenlist==1.3.3
!pip install huggingface-hub==0.13.4
!pip install importlib-resources==5.12.0
!pip install kiwisolver==1.4.4
!pip install matplotlib==3.7.0
!pip install multidict==6.0.4
!pip install openai==0.27.0
!pip install packaging==23.0
!pip install psutil==5.9.4
!pip install pycocotools==2.0.6
!pip install pyparsing==3.0.9
!pip install python-dateutil==2.8.2
!pip install pyyaml==6.0
!pip install regex==2022.10.31
!pip install tokenizers==0.13.2
!pip install tqdm==4.64.1
!pip install transformers==4.28.0
!pip install torchvision
!pip install timm==0.6.13
!pip install spacy==3.5.1
!pip install webdataset==0.2.48
!pip install scikit-learn==1.2.2
!pip install scipy==1.10.1
!pip install yarl==1.8.2
!pip install zipp==3.14.0
!pip install omegaconf==2.3.0
!pip install opencv-python==4.7.0.72
!pip install iopath==0.1.10
!pip install decord==0.6.0
!pip install tenacity==8.2.2
!pip install peft
!pip install pycocoevalcap
!pip install sentence-transformers
!pip install umap-learn
!pip install notebook
!pip install gradio==3.24.1
!pip install gradio-client==0.0.8
!pip install wandb

  Using cached cchardet-2.1.7.tar.gz (653 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-311
      creating build/lib.linux-x86_64-cpython-311/cchardet
      copying src/cchardet/version.py -> build/lib.linux-x86_64-cpython-311/cchardet
      copying src/cchardet/__init__.py -> build/lib.linux-x86_64-cpython-311/cchardet
      running build_ext
      building 'cchardet._cchardet' extension
      creating build/temp.linux-x86_64-cpython-311
      creating build/temp.linux-x86_64-cpython-311/src
      creating build/temp.linux-x86_64-cpython-311/src/cchardet
      creating build/temp.linux-x86_64-cpython-311/src/ext
      creating build/temp.linux-x86_64-cpython-311/src/ext/uchardet
      creating build/temp.linux-x

In [3]:
from minigpt4.processors.blip_processors import Blip2ImageEvalProcessor
from customized_mini_gpt4 import CustomizedMiniGPT4
import torch

# モデルとトークナイザーの準備
model = CustomizedMiniGPT4(
    gpt_neox_model="rinna/bilingual-gpt-neox-4b"
)
tokenizer = model.gpt_neox_tokenizer
if torch.cuda.is_available():
    # model = model.to("cuda")
    model = model.to("cpu").half()  # データ型を16ビット浮動小数点数に変換

# チェックポイントの読み込み
ckpt = torch.load("./checkpoint.pth", map_location="cpu")
model.load_state_dict(ckpt['model'], strict=False)

# VISプロセッサの準備
vis_processor = Blip2ImageEvalProcessor()

/home/mhiavio/anaconda3/envs/rinna/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading VIT
freeze vision encoder
Loading VIT Done
Loading Q-Former
freeze Qformer
Loading Q-Former Done
Loading LLM
Loading LLM Done


In [4]:
# プロンプトの準備
prompt = [
    {
        "speaker": "ユーザー",
        "text": "<Img><ImageHere></Img>画像を説明して"
    },
]
prompt = [
    f"{uttr['speaker']}: {uttr['text']}"
    for uttr in prompt
]
prompt = "\n".join(prompt)
prompt = (
    prompt
    + "\n"
    + "システム: "
)
print(prompt)

ユーザー: <Img><ImageHere></Img>画像を説明して
システム: 


In [5]:
import requests
from PIL import Image

# 画像の埋め込みの準備
image_url = "https://huggingface.co/rinna/bilingual-gpt-neox-4b-minigpt4/resolve/main/sample.jpg"
raw_image = Image.open(requests.get(image_url, stream=True).raw).convert("RGB")
image = vis_processor(raw_image).unsqueeze(0).to(model.device)
image_emb = model.encode_img(image)

RuntimeError: Input type (float) and bias type (c10::Half) should be the same

In [ ]:
# 入力埋め込みの準備
embs = model.get_context_emb(prompt, [image_emb])

In [ ]:
# 推論の実行
output_ids = model.gpt_neox_model.generate(
    inputs_embeds=embs,
    max_new_tokens=512,
    do_sample=True,
    temperature=1.0,
    top_p=0.85,
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id
)
output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
print(output)
raw_image